# Self Organizing Map for Data Visualization and Clustering

## Introduction
A Self Organizing Map (SOM) is a type of neural network that is trained using unsupervised learning. In contrast to a standard neural network a SOM only has input and output layer (no hidden layer), and is trained using competetive learning (not minimizing error using backpropogation). It is used to visualize high dimensional data in lower dimensions (usually two dimensions) and classify observations into clusters. SOM preserve the topology of the training data when mapping it to a lower demensional space, hence the grouping patterns in the lower demsional data will be consistent to the original training data. 

SOM methadology has similarities to Principal Components Analysis (PCA is used for demensionality reduction) and K-Means (used for clustering). In K-means the initialized centroids get updated over itterations independantly, but in SOM a subset of the centroids get updated together (more on this below). In PCA the orginal training data is projected onto a linear surface (line, plane, hyperplane), but this linear projection constraint does not apply to SOM.

## Application of Self Organizing Maps

## Problem and Data

## Algorithm to implement Self Organizing Map